In [ ]:
# Question 3

using Statistics
using CSV
using DataFrames
using LinearAlgebra
using Dates


In [ ]:
# Question 4
filepath = "C:/Users/Matt/Downloads/4.5_month_earthquakes.csv"
df = CSV.read(filepath, DataFrame, dateformat="dd-mm-yyyy")
df2 = CSV.read(filepath, DataFrame, dateformat="dd-mm-yyyy", limit = 10)

In [ ]:
# Question 5
magnitudes = df.mag
println("Min magnitude: [$(minimum(magnitudes))]")
println("Max magnitude: [$(maximum(magnitudes))]")
println("Average magnitude: [$(sum(magnitudes) / length(magnitudes))]")

In [ ]:
# Question 6
for x in 1:length(df.time)
    df.time[x] = df.time[x][1:19]
end
for x in 1:10
    println("$(df.time[x])")
end

In [ ]:
# Question 7

function inTimeRange(N, i)
    present = Dates.now()
    # println((Dates.value(present) - Dates.value(DateTime(df.time[x], DateFormat("yyyy-mm-ddTHH:MM:SS")))))
    dayPassed = ((Dates.value(present) - Dates.value(DateTime(df.time[i], DateFormat("yyyy-mm-ddTHH:MM:SS.sssZ"))))  / (1000 * 3600 * 24))
    # println("the day passed for index $i is $dayPassed")
    return dayPassed <= N
end

function avgMagnitudeNDays(N)  
    sumOfMag = 0
    i = 1
    count = 0
    for x in eachrow(df)
        # println("at index $i")
        if (inTimeRange(N, i))
            sumOfMag += df.mag[i]
            count += 1
        end
        i += 1
    end
    return (sumOfMag / count)
end

# avgMagnitudeNDays(20)

for x in [10, 20, 30, 40 ,50] 
    res = avgMagnitudeNDays(x)
    println("This is in $x days")
    println(res)
end


In [ ]:
# Question 8
function convertNToDegree(n) #n is in km
    return (n /= 100)
end

function avgMagnitudeWithinNKmOfLatLong(N, latitude, longitude)
    n = convertNToDegree(N)
    sumOfMag = 0
    count = 0
    for x in eachrow(df) 
        distance = sqrt((x.latitude - latitude)^2 + (x.longitude - longitude)^2)
        if (distance < n)
            sumOfMag +=  x.mag
            count += 1
        end
    end
    return sumOfMag / count
end

println(avgMagnitudeWithinNKmOfLatLong(2000, 29, 14))
println(avgMagnitudeWithinNKmOfLatLong(65000, 127, 123))
println(avgMagnitudeWithinNKmOfLatLong(3000, 10, 142))
println(avgMagnitudeWithinNKmOfLatLong(3000, -47, 189))

In [ ]:
# Problem 9

# function filtering out the regions that are in the radius   
function isInRadius(row, N, latitude, longitude)
    n = convertNToDegree(N)
    distance = sqrt((row.latitude - latitude)^2 + (row.longitude - longitude)^2)
    if (distance < n) # the place is in the radius
        return true
    end
    return false
end

# function to find the highest magnitude region among the ones in the radius
function highestMagnitude(df)
    highestMagnitude = 0
    result = ""
    for x in eachrow(df) 
        if x.mag > highestMagnitude
            highestMagnitude = x.mag
            result = x.place
        end
    end
    return result
end
function mostActiveRegion(N, latitude , longitude)
    # searching for the most active region
    df_modified = filter(row -> isInRadius(row, N, latitude, longitude), df)
    #finding the max maginitude region inside the radius and return its name
    df_mod2 = combine(groupby(df_modified, :place), :mag => sum => :mag)
    
    mostActiveRegion = highestMagnitude(df_mod2)
    return mostActiveRegion
end

# printing results
result = mostActiveRegion(500, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(1000, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(2000, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(3000, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(4000, -23.7222, -179.4605)
println(result)

In [ ]:
# Problem 10
function inTimeRange(M, x) #modified for date value instead of dataframe index for input
    present = Dates.now()
    # println((Dates.value(present) - Dates.value(DateTime(df.time[x], DateFormat("yyyy-mm-ddTHH:MM:SS")))))
    dayPassed = ((Dates.value(present) - Dates.value(DateTime(x, DateFormat("yyyy-mm-ddTHH:MM:SS.sssZ"))))  / (1000 * 3600 * 24))
    # println("the day passed for index $i is $dayPassed")
    if (dayPassed <= M)
        return true
    end
    return false
end

# modified for inTimeRange itegration
function highestMagnitude(df, M)
    highestMagnitude = 0
    result = ""
    for x in eachrow(df) 
        if (x.mag > highestMagnitude) && (inTimeRange(M, x.time))
            highestMagnitude = x.mag
            result = x.place
        end
    end
    return result
end

function mostActiveRegion(N, M, latitude , longitude)
    # searching for the most active region
    df_modified = filter(row -> isInRadius(row, N, latitude, longitude), df)
    df_mod3 = filter(row -> inTimeRange(M, row.time), df_modified)
    #finding the max maginitude region inside the radius and return its name
    df_mod4 = combine(groupby(df_mod3, :place), :mag => sum => :mag)
    mostActiveRegion = highestMagnitude(df_mod4)
    return mostActiveRegion
end



#printing results
result = mostActiveRegion(500, 10, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(500, 20, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(500, 30, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(1000, 10, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(1000, 20, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(1000, 30, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(2000, 10, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(2000, 20, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(2000, 30, -23.7222, -179.4605)
println(result)


TU PHAM

MATT LE

MINH LE